In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def one_mcvis_euclidean(x_sample):
    x_sample_mean = x_sample.mean(axis=0, keepdims=True)
    x_sample_centred = x_sample - x_sample_mean  # X2 in R
    s = np.sqrt(np.sum(x_sample_centred ** 2, axis=0))  # s in R
    Z = x_sample_centred / s  # Z in R
    x_norm = np.sqrt(np.sum(x_sample ** 2, axis=0))  # x_norm in R
    v = x_norm / s  # v in R
    D = np.diag(v)  # D in R
    Z1 = np.matmul(Z, D)  # Z1 in R
    crossprodZ1 = np.matmul(np.transpose(Z1), Z1)
    eigens = np.linalg.svd(crossprodZ1, compute_uv=False)  # Not in R
    v2 = 1 / eigens  # Not in R
    vif = np.diag(np.linalg.inv(crossprodZ1))  # Not in R
    result = dict();
    result["v2"] = v2
    result["vif"] = vif

    return(result)


def one_mcvis_none(x_sample):
    crossprodX1 = np.matmul(np.transpose(x_sample), x_sample)
    eigens = np.linalg.svd(crossprodX1, compute_uv=False)  # Not in R
    v2 = 1 / eigens  # Not in R
    vif = np.diag(np.linalg.inv(crossprodX1))  # Not in R
    result = dict();
    result["v2"] = v2
    result["vif"] = vif

    return(result)

def mcvisPy(x, standardise_method = "euclidean"):
    n = x.shape[0]
    p = x.shape[1]

    #    print("n is ", n)
    #    print("p is ", p)
    #    print("x is \n", np.round(x, 2))
    #    print("Modified x's first column is \n", np.round(x[:, 0], 2))
    #    print("Correlation matrix of x", np.corrcoef(x, rowvar = False))

    nexp = 1000
    v2_mat = np.ones((p, nexp))
    vif_mat = np.ones((p, nexp))

    for i in range(nexp):
        index = np.random.choice(range(n), n)
        x_sample = x[index, :]  # X1 in R
        
        if standardise_method == "euclidean": 
            out = one_mcvis_euclidean(x_sample)
        else standardise_method == "none":
            out = one_mcvis_none(x_sample)

        v2_mat[:, i] = out["v2"]  # v2 in R
        vif_mat[:, i] = out["vif"]  # vif in R

    index_list = list(chunks(range(1000), 100))  # indexList in R
    tstat_mat = np.ones((p, 10))
    tor = np.ones((p, p))

    for j in range(p):
        for this_index in range(10):
            lmy = v2_mat[j, index_list[this_index]]
            lmx = sm.add_constant(np.transpose(vif_mat[:, index_list[this_index]]))
            lm_obj = sm.OLS(lmy, lmx)
            lm_result = lm_obj.fit()
            lm_tvalues = lm_result.tvalues
            tstat_mat[:, this_index] = lm_tvalues[1:]
        tor[j, :] = np.mean(tstat_mat ** 2, axis=1)

    tor_rowsums = np.sum(tor, axis=1, keepdims=True)  # not in R
    mc = tor / tor_rowsums
    mc_rownames = range(p + 1)[1:]

    mc_pd = pd.DataFrame(mc,
                         columns=mc_rownames,
                         index=mc_rownames[::-1])

    result = dict()
    result["mc_pd"] = mc_pd
    result["t_square"] = tor

    return result

SyntaxError: invalid syntax (<ipython-input-16-40ae6d5a0f4f>, line 62)

In [17]:
p = 5
n = 20

x = np.random.normal(0, 1, n * p).reshape(n, p)
x[:, 0] = x[:, 1] + np.random.normal(0, 0.5, n)
index = np.random.choice(range(n), n)
x_sample = x[index, :]  # X1 in R

In [19]:
out = one_mcvis_euclidean(x_sample)
print(out)
print(out["v2"])
print(out["vif"])

{'v2': array([ 0.2795075 ,  0.87662821,  2.53034324,  4.63085754, 16.33564058]), 'vif': array([7.32840434, 8.58596245, 2.75274495, 3.15254421, 2.83332112])}
[ 0.2795075   0.87662821  2.53034324  4.63085754 16.33564058]
[7.32840434 8.58596245 2.75274495 3.15254421 2.83332112]


In [20]:
out = one_mcvis_none(x_sample)
print(out)
print(out["v2"])
print(out["vif"])

{'v2': array([0.0137303 , 0.04986985, 0.09520612, 0.21499601, 0.98876779]), 'vif': array([0.38940767, 0.59266957, 0.06702637, 0.16388476, 0.14958171])}
[0.0137303  0.04986985 0.09520612 0.21499601 0.98876779]
[0.38940767 0.59266957 0.06702637 0.16388476 0.14958171]


In [21]:
out = mcvisPy(x, standardise_method = "euclidean")

In [22]:
print(out)

{'mc_pd':           1         2         3         4         5
5  0.161551  0.132891  0.304449  0.339780  0.061330
4  0.111188  0.077066  0.302678  0.213782  0.295286
3  0.113628  0.127386  0.250127  0.446911  0.061949
2  0.039996  0.041407  0.167742  0.500507  0.250347
1  0.314710  0.588246  0.040957  0.040665  0.015422, 't_square': array([[  3.8890257 ,   3.19910247,   7.32902632,   8.17954845,
          1.47640072],
       [  2.25008262,   1.55955701,   6.12520533,   4.32623104,
          5.97559809],
       [  4.81115195,   5.39368064,  10.59069721,  18.92281779,
          2.62301274],
       [  5.28476875,   5.4712215 ,  22.16402225,  66.13280667,
         33.07882696],
       [202.29917365, 378.13130228,  26.32775797,  26.14003311,
          9.91345524]])}


In [24]:
out = mcvisPy(x, standardise_method = "none")

In [25]:
out

{'mc_pd':           1         2         3         4         5
 5  0.182069  0.189206  0.237770  0.088708  0.302248
 4  0.146218  0.207032  0.363126  0.154469  0.129156
 3  0.140844  0.167583  0.063800  0.568503  0.059270
 2  0.049279  0.062686  0.104042  0.506842  0.277151
 1  0.198582  0.769031  0.004236  0.011310  0.016842,
 't_square': array([[1.29953074e+00, 1.35047397e+00, 1.69710199e+00, 6.33158585e-01,
         2.15732466e+00],
        [2.46786669e+00, 3.49429831e+00, 6.12884090e+00, 2.60712434e+00,
         2.17988910e+00],
        [3.75177583e+00, 4.46405034e+00, 1.69948764e+00, 1.51436530e+01,
         1.57882625e+00],
        [8.71135833e+00, 1.10812329e+01, 1.83920661e+01, 8.95968615e+01,
         4.89933779e+01],
        [3.60663217e+02, 1.39671155e+03, 7.69426641e+00, 2.05404678e+01,
         3.05878302e+01]])}